In [2]:
import numpy as np
import warnings
import copy
import os
import pandas as pd
warnings.filterwarnings("ignore")
import sys
sys.path.append('../induction_motor_anomaly_detection/')
import modules,scaler

In [3]:
normal_data = pd.read_csv('../data/combined_data.csv')
anomalous_data = pd.read_csv('../anomalous_data/anomalous_data.csv')


In [4]:
train_data_features = modules.ElectricalFeatureExtractor(current_data=normal_data)
test_data_features = modules.ElectricalFeatureExtractor(current_data=anomalous_data)
train_scaler = scaler.Scaler()
train_scaler.fit_unlabelled_data(train_data_features.feature_dataframe[0])
train_scaled = train_scaler.transform(train_data_features.feature_dataframe[0])
test_scaled = train_scaler.transform(test_data_features.feature_dataframe[0])

using MinMaxScaler()


In [13]:
import tensorflow as tf
import numpy as np

class Autoencoder:
    def __init__(self, input_dim, encoding_dim=64, learning_rate=0.001):
        self.input_dim = input_dim
        self.encoding_dim = encoding_dim
        self.learning_rate = learning_rate
        self.encoder = None
        self.decoder = None
        self._build_graph()

    def _build_graph(self):
        self.X = tf.keras.Input(shape=(self.input_dim,))
        self.hidden_layer = tf.keras.layers.Dense(units=self.encoding_dim, activation=tf.nn.relu)(self.X)
        self.outputs = tf.keras.layers.Dense(units=self.input_dim)(self.hidden_layer)
        self.loss = tf.reduce_mean(tf.square(self.X - self.outputs))
        self.optimizer = tf.optimizers.Adam(learning_rate=self.learning_rate)

    def partial_fit(self, X):
        with tf.GradientTape() as tape:
            loss = self.loss_function(X, training=True)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss.numpy()

    def encode(self, X):
        return self.encoder.predict(X)

    def decode(self, X):
        return self.decoder.predict(X)

    def fit(self, X, epochs=10, batch_size=32):
        self.encoder = tf.keras.models.Model(inputs=self.X, outputs=self.hidden_layer)
        self.decoder = tf.keras.models.Model(inputs=self.encoder.output, outputs=self.outputs)
        self.trainable_variables = self.encoder.trainable_variables + self.decoder.trainable_variables
        n_samples = X.shape[0]
        for epoch in range(epochs):
            avg_loss = 0
            for i in range(0, n_samples, batch_size):
                batch = X[i:i+batch_size]
                loss = self.partial_fit(batch)
                avg_loss += loss * len(batch)
            avg_loss /= n_samples
            print("Epoch:", epoch+1, "loss:", avg_loss)


In [ ]:
autoencoder = Autoencoder(input_dim=25)
errors = []
# Online learning loop
for i in test_scaled:
    # Get new data
    X_new =  i
    
    # Partial fit with new data
    loss = autoencoder.partial_fit(X_new)
    
    # Check for anomalies
    encoded_X = autoencoder.encode(X_new)
    decoded_X = autoencoder.decode(encoded_X)
    errors = np.mean(np.square(X_new - decoded_X), axis=1)
    # is_anomaly = errors > threshold
